In [2]:
#importing libraries
import numpy as np 
import pandas as pd 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
#  Load of the data 
train_class = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
sample_sub = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
train_jpg_directory = '/kaggle/input/planets-dataset/planet/planet/train-jpg'
test_jpg_directrory = '/kaggle/input/planets-dataset/planet/planet/test-jpg'


In [4]:
train_class.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
sample_sub.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [6]:
#define a function to split the tags and store a set of the tags in a variable called labels.
def splitting_tags(tags):
    global labels
    for tag in tags.split():
        labels.add(tag)

# Initialize the set of unique labels
labels = set()

train_class1 = train_class.copy()
# Apply the splitting_tags function to the 'tags' column
train_class1['tags'].apply(splitting_tags)

# Convert the set of unique labels to a list
labels = list(labels)

print(labels)

['blow_down', 'cloudy', 'selective_logging', 'clear', 'cultivation', 'artisinal_mine', 'road', 'blooming', 'conventional_mine', 'bare_ground', 'partly_cloudy', 'haze', 'slash_burn', 'habitation', 'water', 'agriculture', 'primary']


In [7]:
# Assert that the length of the DataFrame is the same as the number of rows in the shape
assert len(train_class1['image_name'].unique()) == train_class1.shape[0]

print("Assertion passed.")

Assertion passed.


In [8]:
##One hot encoding is performed on the labels in train classes
for tag in labels:
    train_class1[tag] = train_class1['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
## adding .jpg extension to the column image_name so as to have same name format as the image files
train_class1['image_name'] = train_class1['image_name'].apply(lambda x: '{}.jpg'.format(x))
train_class1.head()

,image_name,tags,blow_down,cloudy,selective_logging,clear,cultivation,artisinal_mine,road,blooming,conventional_mine,bare_ground,partly_cloudy,haze,slash_burn,habitation,water,agriculture,primary
0,train_0.jpg,haze primary,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,train_1.jpg,agriculture clear primary water,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1
2,train_2.jpg,clear primary,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,train_3.jpg,clear primary,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,train_4.jpg,agriculture clear habitation primary road,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1


In [9]:
# Define the newly added label columns
new_columns = list(train_class1.columns[2:])

# Print the newly added label columns
print("New Label Columns:", new_columns)

New Label Columns: ['blow_down', 'cloudy', 'selective_logging', 'clear', 'cultivation', 'artisinal_mine', 'road', 'blooming', 'conventional_mine', 'bare_ground', 'partly_cloudy', 'haze', 'slash_burn', 'habitation', 'water', 'agriculture', 'primary']


In [10]:
new_columns

['blow_down',
 'cloudy',
 'selective_logging',
 'clear',
 'cultivation',
 'artisinal_mine',
 'road',
 'blooming',
 'conventional_mine',
 'bare_ground',
 'partly_cloudy',
 'haze',
 'slash_burn',
 'habitation',
 'water',
 'agriculture',
 'primary']

In [11]:
#define a function for fbeta scoring
def fbeta(y_true, y_pred, beta=2, epsilon=1e-4):
    beta_squared = beta ** 2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis=1)
    fp = tf.reduce_sum(y_pred, axis=1) - tp
    fn = tf.reduce_sum(y_true, axis=1) - tp
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    fbeta = (1 + beta_squared) * precision * recall / (beta_squared * precision + recall + epsilon)
    return fbeta

In [12]:
#define a function for accuracy for multi_label classification
import tensorflow as tf

def multi_label_acc(y_true, y_pred, epsilon=1e-4):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis=1)
    fp = tf.reduce_sum(y_pred, axis=1) - tp
    fn = tf.reduce_sum(y_true, axis=1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
    
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32) * tf.cast(tf.logical_not(y_pred), tf.float32), axis=1)
    
    accuracy = (tp + tn) / (tp + tn + fp + fn + epsilon)
    return accuracy


In [13]:
from tensorflow.keras.metrics import MeanIoU

def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))
    
    opt = Adam(lr=1e-4)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=[fbeta, multi_label_acc])
    
    return model


In [14]:
# Define the filepath for saving the best model
checkpoint_filepath = 'best_model.h5'

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(checkpoint_filepath, 
                             monitor='val_fbeta', 
                             mode='max', 
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1)

In [15]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

In [16]:
train_generator = train_image_gen.flow_from_dataframe(dataframe=train_class1,
                                                directory =train_jpg_directory,  
                                                x_col="image_name", y_col=new_columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 32384 validated image filenames.


In [17]:
#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train_class1,
                                                directory = train_jpg_directory,  
                                                x_col="image_name", y_col=new_columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 8095 validated image filenames.


In [18]:
#initializing the model
model1 = build_model()

#this shows the summary of the model, simply put as the model architecture
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        1

In [19]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_fbeta', mode = 'max',
                                       save_best_only = True, save_weights_only = True)

In [22]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, steps_per_epoch = step_train_size, validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 5, 
            callbacks = [save_best_check_point])

Epoch 1/5
2024/2024 [==============================] - 2428s 1s/step - loss: 0.1800 - fbeta: 0.7458 - multi_label_acc: 0.9296 - val_loss: 0.1567 - val_fbeta: 0.7759 - val_multi_label_acc: 0.9379
Epoch 2/5
2024/2024 [==============================] - 2403s 1s/step - loss: 0.1650 - fbeta: 0.7734 - multi_label_acc: 0.9345 - val_loss: 0.1471 - val_fbeta: 0.7913 - val_multi_label_acc: 0.9412
Epoch 3/5
2024/2024 [==============================] - 2443s 1s/step - loss: 0.1575 - fbeta: 0.7871 - multi_label_acc: 0.9374 - val_loss: 0.1456 - val_fbeta: 0.7848 - val_multi_label_acc: 0.9406
Epoch 4/5
2024/2024 [==============================] - 2414s 1s/step - loss: 0.1512 - fbeta: 0.7988 - multi_label_acc: 0.9398 - val_loss: 0.1509 - val_fbeta: 0.7928 - val_multi_label_acc: 0.9399
Epoch 5/5
2024/2024 [==============================] - 2440s 1s/step - loss: 0.1458 - fbeta: 0.8112 - multi_label_acc: 0.9421 - val_loss: 0.1371 - val_fbeta: 0.8048 - val_multi_label_acc: 0.9434


In [23]:
#initializing a second model so we can make predictions
model2 = build_model()

##adding .jpg extension to image name in the sample submission file
sample_sub1 = sample_sub.copy()
sample_sub1['image_name'] = sample_sub1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_sub1.head()

,image_name,tags
0,test_0.jpg,primary clear agriculture road water
1,test_1.jpg,primary clear agriculture road water
2,test_2.jpg,primary clear agriculture road water
3,test_3.jpg,primary clear agriculture road water
4,test_4.jpg,primary clear agriculture road water


In [24]:
#loading in the weights of the trained model so we can make predictions with it
model2.load_weights('best_model.hdf5')

#we divide the sample submission file into two splits, first test1_df which contains the first 40669 images 
test1_df = sample_sub1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test1_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [29]:
#initializing imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)


#creating a generator for the images found in the first test image files
test_generator1 = test_image_gen.flow_from_dataframe(dataframe=test1_df, 
                                                directory=test_jpg_directrory, 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size1 = int(np.ceil(test_generator1.samples/test_generator1.batch_size))

Found 40669 validated image filenames.


In [30]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator1.reset()
pred1 = model2.predict(test_generator1, steps = step_test_size1, verbose = 1)

2542/2542 [==============================] - 606s 238ms/step


In [31]:
#this is to get the filenames in the generator using the attribute .filenames
file_names1 = test_generator1.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
result1.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,clear primary
4,test_4.jpg,partly_cloudy primary


In [32]:
#second batch of the test dataset
test2_df = sample_sub1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
test2_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [33]:
#creating a generator for the second batch of test image files
test_generator2 = test_image_gen.flow_from_dataframe(dataframe=test2_df, 
                                                directory="../input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size2 = int(np.ceil(test_generator2.samples/test_generator2.batch_size))

Found 20522 validated image filenames.


In [34]:
#we reset the generator to avoid shuffling, then make prediction on the generator
test_generator2.reset()
pred2 = model2.predict(test_generator2, steps = step_test_size2, verbose = 1)

1283/1283 [==============================] - 303s 236ms/step


In [35]:
#this is to get the filenames in the generator using the attribute .filenames
file_names2 = test_generator2.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names2, 'tags': pred_tags2})
result2.head()

,image_name,tags
0,file_0.jpg,clearprimary
1,file_1.jpg,roadpartly_cloudyagricultureprimary
2,file_10.jpg,hazewaterprimary
3,file_100.jpg,clearprimary
4,file_1000.jpg,clearprimary


In [36]:
#for the final result of the predicted tags for the test images, we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

(61191, 2)


,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,clear primary
4,test_4.jpg,partly_cloudy primary


In [37]:
#we need to remove the .jpg extension from the image_name of the last_result because from the sample submission the 
#extension was not there, we added it for easy manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,clear primary
4,test_4,partly_cloudy primary


In [38]:
#Finally, we save the result to a csv file using the .to_csv() method and setting the index to false.
last_result.to_csv('sub.csv', index = False)